In [4]:
acceptable_distance = 10

In [8]:
import numpy as np
from shapely.geometry import Point, LineString


class TheRightRoute():
    def __init__(self, waypoints: LineString, acceptable_distance):
        self.route = waypoints
        self.acceptable_distance = acceptable_distance
        
        self.start_pt = Point(self.route.coords[0])
        self.finish_line = self.calc_finish_line()
    
    def calc_finish_line(self):
        if self.route.coords[-1][0]-self.route.coords[-2][0] == 0:
            RouteEndAngle = np.pi /2
        else:
            RouteEndAngle = np.arctan((self.route.coords[-1][1]-self.route.coords[-2][1])/ \
                                      (self.route.coords[-1][0]-self.route.coords[-2][0]))
        finish_line = LineString([
            Point(
                self.route.coords[-1][0] - acceptable_distance * np.sin(RouteEndAngle),
                self.route.coords[-1][1] + acceptable_distance * np.cos(RouteEndAngle)),
            Point(
                self.route.coords[-1][0] + acceptable_distance * np.sin(RouteEndAngle),
                self.route.coords[-1][1] - acceptable_distance * np.cos(RouteEndAngle))
        ])
        return finish_line
    
    
    
    
    
# RouteToFollow = LineString([(0,0), (60,100)]) # Straight line
RouteToFollow = LineString([(0,0), (60, 0), (60, 60)]) # L-shape
# RouteToFollow = LineString([(0,0), (60, 0), (60, 60), (100, 50), (90, 30), (90, 0), (120,0)]) # Bumpy road
# RouteToFollow = LineString([(0,0), (60, 0), (60, 60), (100, 50), (75, 30), (90, 0), (120,0)]) # Bumpy road with shortcut
# RouteToFollow = LineString([(0,0), (60, 0), (60, 60), (100, 50), (90, 30), (90, 0), (120,0), (130,90), (10, 85), (10, 30)]) # Snail
# RouteToFollow = LineString([(0,0), (20, 0), (20, 30), (50, 30), (50, -30), (20, -30), (20,-60), (80,-60), (80, 60), \
#                             (-30, 60), (-30, 0), (0, -90), (110, -90), (110, 0), (130, 0)]) # Labyrinth
TheRightRoute(RouteToFollow, acceptable_distance)

In [ ]:
class pathfinder():
    def __init__(self):
        